In [1]:
import  Utils
Utils.set_seed(Utils.seed)

import  Dataloader,  Model
import  torch
import  tqdm
import  numpy                as  np
import  matplotlib.pyplot    as  plt
import  torch.nn             as  nn
from    sklearn.metrics import confusion_matrix, classification_report

In [2]:
prediction_input_size   = Utils.input_horizon
epochs                  = Utils.num_epochs
# Simp_Conv().to(device = Utils.Device)#ResNetFeatureExtractor().to(device = Utils.Device)#
model     = Model.Structure_RNN().to(device = Utils.Device)
criterion = torch.nn.CrossEntropyLoss(label_smoothing = 0.3)
optimizer = torch.optim.Adam(
                            model.parameters(),
                            lr=1e-4,
                            # momentum=0.9,
                            weight_decay=1e-3
                            )

In [3]:
loss_t = []
ll = (Dataloader.X_numpy.shape[1]//prediction_input_size)-1
for epoch in range(1,epochs+1):
    print(ll)
    loop_train = tqdm.tqdm(range(ll),total=ll,desc="train",position=0,leave=True)
    loss_train_list = []

    j = 0
    accu = 0
    for Batch in loop_train:#range(Dataloader.X_numpy.shape[0]):
        for test in range(17):#[np.where(Dataloader.Test_ID_numpy[:,0] == data)[0][0] for data in [1,4,5,6,7,9,11,12,19,24,26,27]]:#28,29
            model.train()
            optimizer.zero_grad()
            x = Dataloader.X_numpy[test,Batch*prediction_input_size:(Batch+1)*prediction_input_size]+np.random.random(size=(prediction_input_size))/7
            y = Dataloader.Y_numpy[test,Batch*prediction_input_size: Batch*prediction_input_size+1]
            x = (x-x.mean())/x.std()
            prediction = model.forward(x)

            if torch.argmax(prediction).item() == y:
                accu +=1
            j += 1    
            
            if y==0:
                y = np.array([1,0])
            else:
                y = np.array([0,1])

            loss_train = criterion(prediction, torch.tensor(y.reshape(1,2),dtype=torch.float32).to(device=Utils.Device))
            # Back propagation
            loss_train.backward()
            # Update model parameters
            optimizer.step()
            loss_train_list.append(loss_train)
            
            if Batch%10 == 0:
                # print(loss_train)
                loop_train.set_description(f"{epoch=}- Test : {test} {y=}")
                loop_train.set_postfix(
                    loss_batch="{:.7f}".format(torch.tensor(loss_train_list).mean()),
                    Accu ="{:.5f}".format(100*accu/j),refresh=True, )
                    
        loss_t.append(torch.tensor(loss_train_list).mean())

440


epoch=1- Test : 16 y=array([0, 1]): 100%|██████████| 440/440 [02:54<00:00,  2.52it/s, Accu=84.61853, loss_batch=0.5278431]


In [6]:
accu_li =[]

y_list = []
p_list = []

ll = (Dataloader.X_numpy.shape[1]//prediction_input_size)-1

with torch.inference_mode():
    for test in range(16,20):#[np.where(Dataloader.Test_ID_numpy[:,0] == data)[0][0] for data in [22,21,23,20]]:
        loop_train = tqdm.tqdm(range(ll),total=ll,desc="train",position=0,leave=True)
        j = 0
        accu = 0
        for Batch in loop_train:
            model.eval()

            x = Dataloader.X_numpy[test,Batch*prediction_input_size:(Batch+1)*prediction_input_size]
            y = Dataloader.Y_numpy[test,Batch*prediction_input_size: Batch*prediction_input_size+1]
            v = model(x)
            
            y_list.append(y)
            p_list.append(torch.argmax(v).item())
            
            if torch.argmax(v).item() == y:
                accu +=1

            j += 1
            if Batch%10 == 0:
                loop_train.set_description(f"Test : {test} {y=}")
                loop_train.set_postfix(
                    loss_batch="{:.7f}".format(torch.tensor(loss_train_list).mean()),
                    Accu ="{:.5f}".format(100*accu/j),refresh=True, )
            accu_li.append(100*accu/j)
# print(100*accu/j)

Test : 19 y=array([0.]): 100%|██████████| 440/440 [00:09<00:00, 46.09it/s, Accu=97.67981, loss_batch=0.5260018]


In [8]:


# Convert lists to PyTorch tensors if they are not already
y_true = torch.tensor(y_list)
y_pred = torch.tensor(p_list)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# You can also print a classification report which includes precision, recall, and F1-score.
class_names = ["Class 0", "Class 1"]  # Replace with your class names
report = classification_report(y_true, y_pred, target_names=class_names)

# Print the classification report
print("\nClassification Report:")
print(report)


Confusion Matrix:
[[862  18]
 [  0 880]]

Classification Report:
              precision    recall  f1-score   support

     Class 0       1.00      0.98      0.99       880
     Class 1       0.98      1.00      0.99       880

    accuracy                           0.99      1760
   macro avg       0.99      0.99      0.99      1760
weighted avg       0.99      0.99      0.99      1760



C:\Users\YSNsec\AppData\Local\Temp\ipykernel_14924\2273967091.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  y_true = torch.tensor(y_list)
